In [16]:
from PIL import Image
import numpy as np
from collections import Counter
import heapq

img_path = 'asset/pisang4_grayscale.jpg'
img_gray = Image.open(img_path)

pixels = np.array(img_gray).flatten()

In [17]:
freq = Counter(pixels)

class Node:
    def __init__(self, left=None, right=None, root=None):
        self.left = left
        self.right = right
        self.root = root

    def children(self):
        return (self.left, self.right)

    def __lt__(self, other):
        return self.root < other.root

In [18]:
def build_huffman_tree(frequencies):
    heap = [Node(root=freq, left=symbol) for symbol, freq in frequencies.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        node = Node(left=left, right=right, root=left.root + right.root)
        heapq.heappush(heap, node)
    return heap[0]

In [19]:
def assign_codes(node, prefix="", code={}):
    if isinstance(node.left, Node):
        assign_codes(node.left, prefix + "0", code)
    else:
        code[node.left] = prefix or "0"
    if isinstance(node.right, Node):
        assign_codes(node.right, prefix + "1", code)
    else:
        code[node.right] = prefix or "0"
    return code

In [20]:
huffman_tree = build_huffman_tree(freq)
huffman_codes = assign_codes(huffman_tree)

list(huffman_codes.items())[:10]

[(70, '0000000'),
 (None, '1111111'),
 (139, '0000001'),
 (107, '000001'),
 (106, '000010'),
 (129, '000011'),
 (85, '0001000'),
 (197, '00010010'),
 (62, '0001001100'),
 (47, '00010011010')]

In [21]:
encoded_image = ''.join(huffman_codes[pixel] for pixel in pixels)

original_bits = len(pixels) * 8

encoded_bits = len(encoded_image)

compression_ratio = original_bits / encoded_bits

original_bits, encoded_bits, compression_ratio

(1384448, 1233440, 1.1224283305227656)

In [22]:
def decode_huffman(encoded_str, huffman_codes):
    reverse_huffman_codes = {v: k for k, v in huffman_codes.items()}
    current_code = ''
    decoded_pixels = []
    for bit in encoded_str:
        current_code += bit
        if current_code in reverse_huffman_codes:
            decoded_pixels.append(reverse_huffman_codes[current_code])
            current_code = ''
    return decoded_pixels

decoded_pixels = decode_huffman(encoded_image, huffman_codes)

decoded_image = Image.new('L', img_gray.size)
decoded_image.putdata(decoded_pixels)
decoded_image_path = 'asset/pisang4_compress.jpg'
decoded_image.save(decoded_image_path)

decoded_image_path


'asset/pisang4_compress.jpg'